In [1]:
import numpy as np
import pandas as pd
import psycopg2
import getpass
# for configuring connection 
from configobj import ConfigObj
import os


In [2]:
# Create a database connection using settings from config file
config='../db/config.ini'

# connection info
conn_info = dict()
if os.path.isfile(config):
    config = ConfigObj(config)
    conn_info["sqluser"] = config['username']
    conn_info["sqlpass"] = config['password']
    conn_info["sqlhost"] = config['host']
    conn_info["sqlport"] = config['port']
    conn_info["dbname"] = config['dbname']
    conn_info["schema_name"] = config['schema_name']
else:
    conn_info["sqluser"] = 'postgres'
    conn_info["sqlpass"] = ''
    conn_info["sqlhost"] = '192.168.60.144'
    conn_info["sqlport"] = 6432
    conn_info["dbname"] = 'eicu'
    conn_info["schema_name"] = 'public,eicu_crd'
    
# Connect to the eICU database
print('Database: {}'.format(conn_info['dbname']))
print('Username: {}'.format(conn_info["sqluser"]))
if conn_info["sqlpass"] == '':
    # try connecting without password, i.e. peer or OS authentication
    try:
        if (conn_info["sqlhost"] == '192.168.60.144') & (conn_info["sqlport"]=='6432'):
            con = psycopg2.connect(dbname=conn_info["dbname"],
                                   user=conn_info["sqluser"])            
        else:
            con = psycopg2.connect(dbname=conn_info["dbname"],
                                   host=conn_info["sqlhost"],
                                   port=conn_info["sqlport"],
                                   user=conn_info["sqluser"])
    except:
        conn_info["sqlpass"] = getpass.getpass('Password: ')

        con = psycopg2.connect(dbname=conn_info["dbname"],
                               host=conn_info["sqlhost"],
                               port=conn_info["sqlport"],
                               user=conn_info["sqluser"],
                               password=conn_info["sqlpass"])
query_schema = 'set search_path to ' + conn_info['schema_name'] + ';'

Database: eicu
Username: postgres


In [3]:
from sqlalchemy import create_engine,text
con= create_engine('postgresql://eicu@192.168.60.144:6432/eicu')

In [6]:
datadir = '/home/mei/nas/docker/dataset/EICU/eicu-collaborative-research-database-2.0/'
csv = '/home/mei/nas/docker/thesis/data/csv'

## flat_feature table: 
- select patient who comes from direct and emergency and the stay length in icu between 1 and 14 days

In [9]:
create_table_query =  query_schema +"""

drop materialized view if exists flat cascade;
create materialized view flat as
SELECT DISTINCT patientunitstayid, extract(hour from to_timestamp(unitadmittime24,'HH24:MI:SS')) as hour, gender, age, apacheadmissiondx, admissionweight,dischargeweight, unitdischargelocation,unitdischargestatus
FROM patient
WHERE unitadmitsource IN ('Emergency Department', 'Direct Admit') 
    AND unitdischargeoffset >=1440 
    AND unitdischargeoffset <=20160
group by patientunitstayid, hour
order by patientunitstayid, hour
"""

In [10]:
with con.begin() as connection:
    connection.execute(text(create_table_query))
select_query = "SELECT * FROM flat;"
df_flat = pd.read_sql_query(select_query, con)

In [11]:
df_flat

,patientunitstayid,hour,gender,age,apacheadmissiondx,admissionweight,dischargeweight,unitdischargelocation,unitdischargestatus
0,141168,15.0,Female,70,"Rhythm disturbance (atrial, supraventricular)",84.30,85.8,Death,Expired
1,141265,14.0,Male,67,"CVA, cerebrovascular accident/stroke",100.00,91.8,Floor,Alive
2,141266,21.0,Male,73,"Sepsis, renal/UTI (including bladder)",120.40,112.9,Floor,Alive
3,141276,14.0,Female,59,"Arrest, respiratory (without cardiac arrest)",156.60,156.6,Home,Alive
4,141284,23.0,Male,63,Anemia,NaN,88.5,Floor,Alive
...,...,...,...,...,...,...,...,...,...
68441,3353147,17.0,Male,24,"Overdose, sedatives, hypnotics, antipsychotics...",153.40,151.3,Floor,Alive
68442,3353194,10.0,Female,51,Cardiac arrest (with or without respiratory ar...,63.05,65.8,Death,Expired
68443,3353213,7.0,Female,51,Coma/change in level of consciousness (for hep...,54.40,59.4,Home,Alive
68444,3353226,18.0,Female,79,"Effusions, pleural",58.40,55.1,Death,Expired


##  labels
- unitdischarge location, unitdiscahrges status, actualiculos

In [12]:
create_table_query =  query_schema +"""
drop materialized view if exists labels cascade;
create materialized view labels as
  -- select all the data we need from the apache predictions table, plus patient identifier and hospital identifier
  -- information because we only want to select one episode per patient (more on this later)
  with all_labels as (
    select p.uniquepid, p.patienthealthsystemstayid, apr.patientunitstayid, p.unitvisitnumber,
      apr.predictedhospitalmortality, apr.actualhospitalmortality, apr.predictediculos, apr.actualiculos
      from patient as p
      inner join apachepatientresult as apr
        on p.patientunitstayid = apr.patientunitstayid
      -- only use the most recent apache prediction model and exclude anyone who doesn't have at least 24 hours of data
      where apr.apacheversion = 'IVa' and apr.actualiculos >= 1
    )
  select al.patientunitstayid, al.predictedhospitalmortality, al.actualhospitalmortality,
    al.predictediculos, al.actualiculos
    from all_labels as al
    -- 'selection' is a table which will choose a random hospital stay (the lowest number is fine because the stays
    -- are randomly ordered). In the case of multiple ICU stays within that hospital admission, it will choose the
    -- first ICU stay that satisfies the 24 hours of data requirement. The rationale is that the model should be
    -- applied as soon as there is 24 hours of continuous data within the hospital. This query extracts 89143 stays.
    inner join (
      select p.uniquepid, p.patienthealthsystemstayid, min(p.unitvisitnumber) as unitvisitnumber
        from patient as p
        inner join (
          select uniquepid, min(patienthealthsystemstayid) as patienthealthsystemstayid
            from all_labels
            group by uniquepid
          ) as intermediate_selection
          on p.patienthealthsystemstayid = intermediate_selection.patienthealthsystemstayid
        group by p.uniquepid, p.patienthealthsystemstayid
      ) as selection
      on al.patienthealthsystemstayid = selection.patienthealthsystemstayid
      and al.unitvisitnumber = selection.unitvisitnumber;
    inner join flat as f on la.patientunitstayid = f.patientunitstayid;
"""

In [13]:
select_query = "SELECT * FROM labels;"
df_labels = pd.read_sql_query(select_query, con)

In [14]:
df_labels

,patientunitstayid,predictedhospitalmortality,actualhospitalmortality,predictediculos,actualiculos
0,141168,5.9098619924959253E-2,EXPIRED,3.091127,2.4972
1,141194,0.1022830270275107,ALIVE,4.167129,3.3423
2,141203,0.4709733472175876,ALIVE,8.670299,1.2979
3,141227,0.48856191407785921,ALIVE,8.209624,1.1472
4,141233,3.5490436381315972E-2,ALIVE,3.107540,10.8923
...,...,...,...,...,...
89138,3353147,4.8307271745665827E-2,ALIVE,5.498694,1.0888
89139,3353190,-1,ALIVE,-1.000000,2.1187
89140,3353194,0.86952467986787907,EXPIRED,7.674351,2.4930
89141,3353213,5.2999142347007012E-2,ALIVE,5.801727,3.4118


In [15]:
flat_index = df_flat.set_index('patientunitstayid').index

In [18]:
df_labels = df_labels[df_labels['patientunitstayid'].isin(flat_index)]
col_labels = ['patientunitstayid','actualiculos']
col_flats = ['patientunitstayid','unitdischargelocation','unitdischargestatus']
df_labels = df_labels[col_labels].merge(df_flat[col_flats], left_on='patientunitstayid', right_on='patientunitstayid', how='inner')

In [19]:
df_labels

,patientunitstayid,actualiculos,unitdischargelocation,unitdischargestatus
0,141168,2.4972,Death,Expired
1,141265,4.2138,Floor,Alive
2,141266,1.0423,Floor,Alive
3,141276,1.1694,Home,Alive
4,141284,1.4416,Floor,Alive
...,...,...,...,...
51332,3353123,1.0409,Death,Expired
51333,3353140,1.3243,Skilled Nursing Facility,Alive
51334,3353147,1.0888,Floor,Alive
51335,3353194,2.4930,Death,Expired


## diagnosis

In [25]:
create_table_query =  query_schema +"""
drop materialized view if exists diagnoses cascade;
create materialized view diagnoses as
  -- for flat:
  
  select f.patientunitstayid, f.apacheadmissiondx  as diagnosisstring
    from flat as f
    -- restrict only to the patients present in the labels materialized view
    inner join labels as l on l.patientunitstayid = f.patientunitstayid
    -- make sure the diagnosis was entered either before the ICU admission, or within the first 24 hours
  
  union
  -- for past medical history:
  select ph.patientunitstayid, ph.pasthistorypath as diagnosisstring
    from pasthistory as ph
    inner join labels as l on l.patientunitstayid = ph.patientunitstayid
    where ph.pasthistoryoffset < 1440
    
"""

In [26]:
with con.begin() as connection:
    connection.execute(text(create_table_query)) 
    
select_query = "SELECT * FROM diagnoses;"
df_dx = pd.read_sql_query(select_query, con)

In [27]:
df_dx

,patientunitstayid,diagnosisstring
0,2275546,"CHF, congestive heart failure"
1,1073584,notes/Progress Notes/Past History/Past History...
2,460254,notes/Progress Notes/Past History/Organ System...
3,1521107,notes/Progress Notes/Past History/Organ System...
4,1435118,notes/Progress Notes/Past History/Organ System...
...,...,...
372381,1620184,"Chest pain, musculoskeletal"
372382,763124,"Sepsis, unknown"
372383,2747847,notes/Progress Notes/Past History/Organ System...
372384,3072067,notes/Progress Notes/Past History/Organ System...
